In [17]:
import os
import sys
import requests
from datetime import datetime

import numpy as np
import pandas as pd
import yfinance as yf

In [18]:
file_name = '../data/price_2.csv'
start_date = '2023-01-02'
# end_date = '2023-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')


def grab_price_data(start_date,end_date):
    #tickers_list = ['JPM', 'COST', 'IBM', 'HD', 'ARWR']
    tickers_list = ['GOOG','IBM']

# Store multiple result sets.
    
    full_price_history = []

    for ticker in tickers_list:
        price_history = yf.Ticker(ticker).history(period='max', start=start_date, end=end_date, interval='1d')

        for index, row in price_history.iterrows():
            row_data = row.to_dict()
            row_data['symbol'] = ticker
            row_data['datetime'] = index.strftime('%Y-%m-%d')  # Convert Pandas Timestamp to datetime string
            full_price_history.append(row_data)


    price_data = pd.DataFrame(full_price_history)
    
    
    price_data_ro = price_data
    price_data = price_data_ro[['datetime', 'symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']] # rearrange column here
    price_data.to_csv('../data/inital_price_data.csv', index=False)
    price_data.head()


if os.path.exists('../data/inital_price_data.csv'):
    
    # Load the data
    price_data = pd.read_csv('../data/inital_price_data.csv')

else:

    # Grab the data and store it.
    grab_price_data(start_date, end_date)

    # Load the data
    price_data = pd.read_csv('../data/inital_price_data.csv')

# Display the head before moving on.
price_data.head()


def calculate_price_change():
    # Sort the data by symbol and datetime
    price_data.sort_values(by = ['symbol','datetime'], inplace = True)

    # calculate the change in price
    price_data['change_in_price'] = price_data['Close'].diff()

    return price_data.head()



def calculate_price_change2():
    mask = price_data['symbol'] != price_data['symbol'].shift(1)

    # For those rows, let's make the value null
    price_data['change_in_price'] = np.where(mask == True, np.nan, price_data['change_in_price'])

    # print the rows that have a null value, should only be 5
    return price_data[price_data.isna().any(axis = 1)]


def smoothed_df():
    # define the number of days out you want to predict
    days_out = 30

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    price_data_smoothed = price_data.groupby(['symbol'])[['Close','Low','High','Open','Volume']].transform(lambda x: x.ewm(span = days_out).mean())

    # Join the smoothed columns with the symbol and datetime column from the old data frame.
    smoothed_df = pd.concat([price_data[['symbol','datetime']], price_data_smoothed], axis=1, sort=False)

    

    # create a new column that will house the flag, and for each group calculate the diff compared to 30 days ago. Then use Numpy to define the sign.
    smoothed_df['Signal_Flag'] = smoothed_df.groupby('symbol')['Close'].transform(lambda x : np.sign(x.diff(days_out)))

    # print the first 50 rows
    return smoothed_df.head(50)



def calculate_RSI():
    # Calculate the 14 day RSI
    n = 14

    # First make a copy of the data frame twice
    up_df, down_df = price_data[['symbol','change_in_price']].copy(), price_data[['symbol','change_in_price']].copy()

    # For up days, if the change is less than 0 set to 0.
    up_df.loc['change_in_price'] = up_df.loc[(up_df['change_in_price'] < 0), 'change_in_price'] = 0

    # For down days, if the change is greater than 0 set to 0.
    down_df.loc['change_in_price'] = down_df.loc[(down_df['change_in_price'] > 0), 'change_in_price'] = 0

    # We need change in price to be absolute.
    down_df['change_in_price'] = down_df['change_in_price'].abs()

    # Calculate the EWMA (Exponential Weighted Moving Average), meaning older values are given less weight compared to newer values.
    ewma_up = up_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())
    ewma_down = down_df.groupby('symbol')['change_in_price'].transform(lambda x: x.ewm(span = n).mean())

    # Calculate the Relative Strength
    relative_strength = ewma_up / ewma_down

    # Calculate the Relative Strength Index
    relative_strength_index = 100.0 - (100.0 / (1.0 + relative_strength))

    # Add the info to the data frame.
    price_data['down_days'] = down_df['change_in_price']
    price_data['up_days'] = up_df['change_in_price']
    price_data['RSI'] = relative_strength_index

    # Display the head.
    return price_data.head(30)



def calculate_Stoc_Osc():
    # Calculate the Stochastic Oscillator
    n = 14

    # Make a copy of the high and low column.
    low_14, high_14 = price_data[['symbol','Low']].copy(), price_data[['symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate the Stochastic Oscillator.
    k_percent = 100 * ((price_data['Close'] - low_14) / (high_14 - low_14))

    # Add the info to the data frame.
    price_data['low_14'] = low_14
    price_data['high_14'] = high_14
    price_data['k_percent'] = k_percent

    # Display the head.
    return price_data.head(30)

def calculate_williams_R():
    # Calculate the Williams %R
    n = 14

    # Make a copy of the high and low column.
    low_14, high_14 = price_data[['symbol','Low']].copy(), price_data[['symbol','High']].copy()

    # Group by symbol, then apply the rolling function and grab the Min and Max.
    low_14 = low_14.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = n).min())
    high_14 = high_14.groupby('symbol')['High'].transform(lambda x: x.rolling(window = n).max())

    # Calculate William %R indicator.
    r_percent = ((high_14 - price_data['Close']) / (high_14 - low_14)) * - 100

    # Add the info to the data frame.
    price_data['r_percent'] = r_percent

    # Display the head.
    return price_data.head(30)


def calculate_MACD():
    # Calculate the MACD
    ema_26 = price_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span = 26).mean())
    ema_12 = price_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span = 12).mean())
    macd = ema_12 - ema_26

    # Calculate the EMA
    ema_9_macd = macd.ewm(span = 9).mean()

    # Store the data in the data frame.
    price_data['MACD'] = macd
    price_data['MACD_EMA'] = ema_9_macd

    # Print the head.
    return price_data.head(30)

def calculate_price_rate_of_change():
    # Calculate the Price Rate of Change
    n = 9

    # Calculate the Rate of Change in the Price, and store it in the Data Frame.
    price_data['Price_Rate_Of_Change'] = price_data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = n))

    # Print the first 30 rows
    return price_data.head(30)

def obv(group):
    
    # Grab the volume and close column.
    volume = group['Volume']
    change = group['Close'].diff()

    # intialize the previous OBV
    prev_obv = 0
    obv_values = []

    # calculate the On Balance Volume
    for i, j in zip(change, volume):

        if i > 0:
            current_obv = prev_obv + j
        elif i < 0:
            current_obv = prev_obv - j
        else:
            current_obv = prev_obv

        # OBV.append(current_OBV)
        prev_obv = current_obv
        obv_values.append(current_obv)
    
    # Return a panda series.
    return pd.Series(obv_values, index = group.index)
        
def calculate_obv():
# apply the function to each group
    obv_groups = price_data.groupby('symbol').apply(obv)

    # add to the data frame, but drop the old index, before adding it.
    price_data['On Balance Volume'] = obv_groups.reset_index(level=0, drop=True)

    # display the data frame.
    return price_data.head(30)

'''
    In this case, let's create an output column that will be 1 if the closing price at time 't' is greater than 't-1' and 0 otherwise.
    In other words, if the today's closing price is greater than yesterday's closing price it would be 1.
'''

# Create a column we wish to predict
def calculate_prediction():
    
# Group by the `Symbol` column, then grab the `Close` column.
    close_groups = price_data.groupby('symbol')['Close']

    # Apply the lambda function which will return -1.0 for down, 1.0 for up and 0.0 for no change.
    close_groups = close_groups.transform(lambda x : np.sign(x.diff()))

    # add the data to the main dataframe.
    price_data['Prediction'] = close_groups

    # for simplicity in later sections I'm going to make a change to our prediction column. To keep this as a binary classifier I'll change flat days and consider them up days.
    price_data.loc[price_data['Prediction'] == 0.0] = 1.0

    # print the head
    return price_data.head(50)
    # OPTIONAL CODE: Dump the data frame to a CSV file to examine the data yourself.
    # price_data.to_csv('final_metrics.csv')



#-----------------------------

calculate_price_change()
calculate_price_change2()
smoothed_df()

calculate_RSI()
calculate_Stoc_Osc()
calculate_williams_R()
calculate_MACD()
calculate_price_rate_of_change()
calculate_obv()
calculate_prediction()


,datetime,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,change_in_price,...,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction
0,2023-01-03,GOOG,89.830002,91.550003,89.019997,89.699997,20738500.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,NaN
1,2023-01-04,GOOG,91.010002,91.239998,87.800003,88.709999,27046500.0,0.0,0.0,-0.989998,...,0.000000,NaN,NaN,NaN,NaN,-0.022211,-0.012340,NaN,-27046500.0,-1.0
2,2023-01-05,GOOG,88.070000,88.209999,86.559998,86.769997,23136100.0,0.0,0.0,-1.940002,...,0.000000,NaN,NaN,NaN,NaN,-0.088461,-0.043537,NaN,-50182600.0,-1.0
3,2023-01-06,GOOG,87.360001,88.470001,85.570000,88.160004,26612600.0,0.0,0.0,1.390007,...,36.435873,NaN,NaN,NaN,NaN,-0.066239,-0.051227,NaN,-23570000.0,1.0
4,2023-01-09,GOOG,89.195000,90.830002,88.580002,88.800003,22996700.0,0.0,0.0,0.639999,...,46.744566,NaN,NaN,NaN,NaN,-0.027303,-0.044110,NaN,-573300.0,1.0
5,2023-01-10,GOOG,86.720001,89.474998,86.699997,89.239998,22855600.0,0.0,0.0,0.439995,...,52.814896,NaN,NaN,NaN,NaN,0.015431,-0.027971,NaN,22282300.0,1.0
6,2023-01-11,GOOG,90.059998,92.449997,89.739998,92.260002,25998800.0,0.0,0.0,3.020004,...,75.201324,NaN,NaN,NaN,NaN,0.179382,0.024504,NaN,48281100.0,1.0
7,2023-01-12,GOOG,92.400002,92.620003,90.570000,91.910004,22754200.0,0.0,0.0,-0.349998,...,70.714929,NaN,NaN,NaN,NaN,0.269786,0.083450,NaN,25526900.0,-1.0
8,2023-01-13,GOOG,91.528000,92.980003,90.930000,92.800003,18630700.0,0.0,0.0,0.889999,...,75.077436,NaN,NaN,NaN,NaN,0.372441,0.150209,NaN,44157600.0,1.0
9,2023-01-17,GOOG,92.779999,92.970001,90.839996,92.160004,22935800.0,0.0,0.0,-0.639999,...,66.818496,NaN,NaN,NaN,NaN,0.406877,0.207717,0.027425,21221800.0,-1.0


In [19]:
print('Before NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Any row that has a `NaN` value will be dropped.
price_data = price_data.dropna()

# Display how much we have left now.
print('After NaN Drop we have {} rows and {} columns'.format(price_data.shape[0], price_data.shape[1]))

# Print the head.
price_data.tail()



Before NaN Drop we have 640 rows and 22 columns
After NaN Drop we have 614 rows and 22 columns


,datetime,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits,change_in_price,...,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume,Prediction
635,2024-04-05,IBM,188.589996,190.320007,188.020004,189.139999,2013600.0,0.0,0.0,1.199997,...,45.044388,187.339996,193.979996,27.108482,-72.891518,0.442773,1.238688,-0.008908,65483800.0,1.0
636,2024-04-08,IBM,189.240005,190.240005,188.910004,189.820007,2678300.0,0.0,0.0,0.680008,...,48.347837,187.339996,193.979996,37.349566,-62.650434,0.388415,1.068633,0.005456,68162100.0,1.0
637,2024-04-09,IBM,190.539993,191.250000,186.660004,189.309998,2794200.0,0.0,0.0,-0.510010,...,45.957159,186.660004,193.979996,36.202142,-63.797858,0.300716,0.915050,0.004297,65367900.0,-1.0
638,2024-04-10,IBM,187.419998,187.919998,185.520004,186.039993,3081700.0,0.0,0.0,-3.270004,...,33.648152,185.520004,193.369995,6.624072,-93.375928,-0.032276,0.725585,-0.024948,62286200.0,-1.0
639,2024-04-11,IBM,186.039993,186.800003,184.580002,185.899994,2860000.0,0.0,0.0,-0.139999,...,33.208764,184.580002,193.279999,15.172328,-84.827672,-0.303968,0.519674,-0.026498,59426200.0,-1.0


In [20]:
price_data.to_csv(file_name)